## Using LLMs to parse the text results in our task !

In [3]:
!pip install requests
!pip install OpenAI



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from openai import OpenAI


# Connect to LM Studio
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="model-identifier",
  messages=[
    {"role": "system", "content": "You are an impartial agent tasked with classifying text answers in a cognitive behavioural task. You live in a Jupyter notebook. You always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message.content)

I'm a language model, don't you fret,
My purpose is to help, and assist you to get,
By answering questions, and providing facts so bright,
I'll do my best to make your day feel just right.

My knowledge is vast, and I'm here to share,
Information on many topics, beyond compare,
From science to art, and everything in between,
I'll do my best to help, with a smile serene.

So if you've got questions, or need some aid,
Just ask me anything, and I'll be glad,
To help you out, and make your day shine bright,
That's what I'm here for, with all my might!


In [ ]:
def llm_request_message(question,subject_answer) :
    
    _general_role = "You are an impartial assistant tasked with classifying subject text responses from a behavioural experiment."
    _experiment_general = "The subjects of the experiment performed actions by placing two points (or arrows) on an unannotated grey screen. They had to learn how to control the level of a gauge using these points."
    _experiment_general_2 =  "The strategy to control the gauge using the points was ambiguous by design. Subjects had to find it by themselves through trial and error."
    _experiment_general_3 = "The gauge was actually controlled by the angle between the horizontal and the vector draw by the points. A 45° angle was optimal."
    _experiment_general_4 = "This means that the optimal strategy was to place the second point should be placed on the upper-right compared to the first point."
    _experiment_general_5 = "Placing the first point on the bottom-left corner and the second point on the top right corner also worked."
    _question = "At the end of the experiment, we asked subjects the following question : **{}**.".format(question["prompt"])
    _task = "Your job is to classify the answer of the participant into one of the following categories :"    
    _categories = "\n".join([cat["letter"] + ". " + cat["content"] for cat in question["categories"]])
    _constraint = "Please only answer using the letter corresponding the the classification of the participant."
    system_directive = "\n".join([_general_role,_experiment_general,_experiment_general_2,_experiment_general_3,_experiment_general_4,_experiment_general_5,_question,_task,_categories,_constraint])
    
    _answer = "Here is the answer of the participant : **{}**".format(subject_answer)
    
    messages=[
            {"role": "system", "content": system_directive},
            {"role": "user", "content": _answer}
        ]
    return messages
    

    # I want you to classify the answer of the participant into one of the following categories : 
    # 1. What was the strategy adopted by the participant ?
    # - A : Did not answer the question
    # - B : Found the optimal strategy (equivalent representations such as "making a 45° angle" are accepted)
    # - C : Found only part of the optimal strategy (e.g. placing points on top of each other or drawing an horizontal line)
    # - D : Thought he/she found the correct strategy but was mistaken
    # - E : Could not find any strategy
    # - F : The answer was off-topic

    # 2. Regardless of what strategy they actually reported, what did the participant suggest he / she should have done instead ?
    # - A : He / She did not answer that question
    # - B : He / She suggested the optimal strategy
    # - C : He / She suggested a strategy that was not optimal

    # Please only answer using the letter corresponding the the classification of the participant.
    # """.format(subject_answer)
    
question_1 = {
        "prompt":"According to you, what was the best strategy in order to control the gauge level ? In retrospect, what other strategy should you have attempted ? (Try to answer as precisely as possible.)",
        "categories":[
            {"letter" : "A", "content" : "The subject did not provide any answer to the question / The answer was off-topic."},
            {"letter" : "B", "content" : "The subject found the optimal strategy."},
            {"letter" : "C", "content" : "The subject found only part of the optimal strategy (e.g. placing points on top of each other, drawing an horizontal line, etc.)."},
            {"letter" : "D", "content" : "The subject thought he/she found the correct strategy but was mistaken."},
            {"letter" : "E", "content" : "The subject did not try any strategy"},
        ]
    }


question_2 = {
        "prompt":"Did you notice any technical issues (buttons not working, graphical bugs, etc.) ? If yes, how did they appear ?",
        "categories":[
            {"letter" : "A", "content" : "The subject did not provide any answer to the question / The answer was off-topic."},
            {"letter" : "B", "content" : "The subject did not encounter any issue."},
            {"letter" : "C", "content" : "The subject encountered only minor technical issues : display delays, lag, stuttering."},
            {"letter" : "D", "content" : "The subject encountered serious issues but only affected a limited amount of data (e.g. only in a few actions)."},
            {"letter" : "E", "content" : "The subject encountered catastrophic issues that impacted the gathering of meaningful data."},
        ]
    }

question_3 = {
        "prompt":"The instructions were meant to help you understand the task before it started. How clear did you find them ? What would you change to make them clearer ?",
        "categories":[
            {"letter" : "A", "content" : "The subject did not provide any answer to the question / The answer was off-topic."},
            {"letter" : "B", "content" : "The subject fully understood the instructions."},
            {"letter" : "C", "content" : "The subject partially understood the instructions."},
            {"letter" : "D", "content" : "The subject struggled to understand the instructions."},
            {"letter" : "E", "content" : "The subject did not understand the instructions at all."},
        ]
    }

question_4 = {
        "prompt":"Do you think there was a logic behind how the gauge responded ? How hard / frustrating did you find the task ? Other remarks ?",
        "categories":[
            {"letter" : "A", "content" : "The subject did not provide any answer to the question / The answer was off-topic."},
            {"letter" : "B", "content" : "The subject believed there was a logic behind how the gauge behaved and found it."},
            {"letter" : "C", "content" : "The subject believed there was a logic behind how the gauge behaved but did not manage to find it."},
            {"letter" : "D", "content" : "The subject believed there was a slight logic behind how the gauge behaved but with significant erratic behaviour."},
            {"letter" : "E", "content" : "The subject did not find any logic in the gauge behaviour."},
        ]
    }

question_5 = {
        "prompt":"Do you think there was a logic behind how the gauge responded ? How hard / frustrating did you find the task ? Other remarks ?",
        "categories":[
            {"letter" : "A", "content" : "The subject did not provide any answer to the question."},
            {"letter" : "B", "content" : "The task was very easy to the subject."},
            {"letter" : "C", "content" : "The task was moderately easy to the subject."},
            {"letter" : "D", "content" : "The task was initially hard but then became easy when the subject found out the correct strategy."},
            {"letter" : "E", "content" : "The task was very hard to the subject."},
        ]
    }

question_6 = {
        "prompt":"Do you think there was a logic behind how the gauge responded ? How hard / frustrating did you find the task ? Other remarks ?",
        "categories":[
            {"letter" : "A", "content" : "The subject did not provide any answer to the question."},
            {"letter" : "B", "content" : "The subject found the task funny / entertaining / interesting."},
            {"letter" : "C", "content" : "The subject reported no strong opinion on the task."},
            {"letter" : "D", "content" : "The subject found the task boring."},
            {"letter" : "E", "content" : "The subject found the task a bit frustrating."},
            {"letter" : "G", "content" : "The subject found the task very frustrating."},
        ]
    }






subject_answer = "the strategy was to start the first point from the bottom left corner to the top right corner in order to get the most out of the guage"
# subject_answer = "What I did was placing the two points at an increasingly higher distance. To maximise that space, I placed them on a diagonal. I randomly/intuitively chose the lower-left corner to upper-right corner (in this sequence), and it appeared to work perfectly, so I did not change the strategy till the end."
subject_answer = "It worked when I put the arrows side by side."
subject_answer = "I could not understand any way to get the gauge to move up. Only towards the end do I think I may have got some control. I tried clicking in so many different areas."
message = llm_request_message(question_1,subject_answer)

print(message)

print(message[0]["content"])
print("---")
print(message[1]["content"])

completion = client.chat.completions.create(
    model="model-identifier",
    messages=message,
    temperature = 0.1,
)

print(completion.choices[0].message.content)


[{'role': 'system', 'content': 'You are an impartial assistant tasked with classifying subject text responses from a behavioural experiment.\nThe subjects of the experiment performed actions by placing two points (or arrows) on an unannotated grey screen. They had to learn how to control the level of a gauge using these points.\nThe strategy to control the gauge using the points was ambiguous by design. Subjects had to find it by themselves through trial and error.\nThe gauge was actually controlled by the angle between the horizontal and the vector draw by the points. A 45° angle was optimal.\nThis means that the optimal strategy was to place the second point should be placed on the upper-right compared to the first point.\nPlacing the first point on the bottom-left corner and the second point on the top right corner also worked.\nAt the end of the experiment, we asked subjects the following question : ***According to you, what was the best strategy in order to control the gauge level

NameError: name 'client' is not defined

In [6]:
()

()